This file contains the code for directly optimizing llama through the algorithms
mentioned in the project report. Due to GPU memory constraint, the code below
requires more than 50G of GPU memory to run, so I did not use this code as part of
project. Still, I include this as a proof of concept that training large models
like LLaMA is possible and illustrate a way to do this.

Attempt was made to run it in 8 bits float precision, but the gradients didn't
work out very well so we reverted to approach 1 and 2 in the paper.

The MCTS part of the code is adopted from https://github.com/suragnair/alpha-zero-general

To run this code, since the action space is different, please change the files in
tictactoe/TicTacToeGame.py, by replacing it with tictactoe/TicTacToeGame_llama.py
and renaming it to TicTacToeGame.py.

In [ ]:
!git lfs install
!git clone https://huggingface.co/tloen/alpaca-lora-7b

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'alpaca-lora-7b'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 48 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (48/48), 5.87 KiB | 600.00 KiB/s, done.


In [ ]:
!git clone https://github.com/tloen/alpaca-lora.git

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 607 (delta 28), reused 34 (delta 19), pack-reused 556
Receiving objects: 100% (607/607), 27.78 MiB | 6.62 MiB/s, done.
Resolving deltas: 100% (360/360), done.


In [ ]:
!pip install -r /content/alpaca-lora/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-h1qhf0cx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-h1qhf0cx
  Resolved https://github.com/huggingface/peft.git to commit 2822398fbe896f25d4dac5e468624dc5fd65a51b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [ ]:
!pip uninstall transformers --yes

Found existing installation: transformers 4.28.1
Uninstalling transformers-4.28.1:
  Successfully uninstalled transformers-4.28.1


In [ ]:
!pip install git+"https://github.com/huggingface/transformers.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-lwcocf8o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-lwcocf8o
  Resolved https://github.com/huggingface/transformers.git to commit 8a817e1ecac6a420b1bdc701fcc33535a3b96ff5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=6972118 sha256=cb8cb97608149bcd442cfc4c6ca715abd395ec934c75ac3e73c70d41cebf3635
  Stored in directory: /tmp/pip-ephem-wheel-cache-vyevrulv/wheels/f7/92/8c/752ff3bfcd3439805d8bbf641614da38ef3226e127ebea86ee
Successfully built transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf", device_map = "sequential")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [ ]:
import sys
import torch
from peft import PeftModel
import transformers

from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
import bitsandbytes as bnb
load_8bit = True

model = LlamaForCausalLM.from_pretrained(
            'decapoda-research/llama-7b-hf',
            # load_in_8bit=load_8bit,
            offload_folder="/content",
            torch_dtype=torch.float32,
            device_map="auto",
        )
# model = PeftModel.from_pretrained(
#     model,
#     "tloen/alpaca-lora-7b",
#     offload_folder="/content",
#     torch_dtype=torch.float16,
# )

/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(instruction, input):
  prompts = []
  for (i, item) in enumerate(input):
    if item:
            output =  f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {instruction}

    ### Input:
    {item}

    ### Response:
    """
    else:
            output = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    {instruction}

    ### Response:
    """
    prompts.append(output)
  return prompts


model.eval()


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): Llama

In [ ]:
generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
    )

In [ ]:
def evaluate(
        instruction,
        input=None,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt",padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=2048,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()


In [ ]:
# with torch.no_grad():
#   print(model(input_ids=input_ids)['logits'][-1].max(-1))

In [ ]:
# Testing
'''
with torch.no_grad():
  generate_ids = model.generate(input_ids=input_ids,max_new_tokens=1)
  for i in generate_ids:
    print(tokenizer.decode(i))
  # print(input_ids)
'''

In [ ]:
value_vec = torch.zeros(32000, dtype=torch.float).to(device)
correctnessvec1 = torch.zeros(32000, dtype=torch.float).to(device)
correctnessvec2 = torch.zeros(32000, dtype=torch.float).to(device)
correctnessvec3 = torch.zeros(32000, dtype=torch.float).to(device)
correctnessvec4 = torch.zeros(32000, dtype=torch.float).to(device)
correctnessvec5 = torch.zeros(32000, dtype=torch.float).to(device)

# Define the indices and values to replace
# indices of tokens 0 to 9
indices = [29900, 29896, 29906, 29941, 29946, 29945, 29953, 29955, 29947, 29929]
# indices of 0
indices1 = [29900]
# indices of '.'
indices2 = [29889]
values = [0.01, 0.15, 0.3, 0.4, 0.45, 0.5, 0.6, 0.7, 0.85, 0.99]

# indices of token A to C
indices_alpha2 = [319,350,315]
# indices of token 1 to 3
indices_numeric = [29896,29906,29941]

correctness1 = torch.ones(1).to(device)
correctness2 = torch.ones(1).to(device)
correctness3 = torch.ones(10).to(device)
correctness4 = torch.ones(3).to(device)
correctness5 = torch.ones(3).to(device)
# Create a tensor of the same shape and data type as the destination tensor
source_tensor = torch.tensor(values, dtype=torch.float).to(device)


# Replace the values at the specified indices
value_vec[indices] = source_tensor
correctnessvec1[indices1] = correctness1
correctnessvec2[indices2] = correctness2
correctnessvec3[indices] = correctness3
correctnessvec4[indices_alpha2] = correctness4
correctnessvec5[indices_numeric] = correctness5

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    StoppingCriteriaList,
    MaxLengthCriteria,
)

In [ ]:
def get_value(board_string):
  prompt = """I am the O player in a game tic-tac-toe, the other player is X and I'm supposed to play next. The board configuration is given in input. If I play optimally and the opponent play optimally, reply a single integer between 0 to 9 representing how likely it is for me of winning, with 0 being least likely and 9 being most likely. No explanations."""
  inputs = generate_prompt(prompt, board_string)
  inputs = tokenizer(inputs, return_tensors="pt", padding=True)
  input_ids = inputs["input_ids"].to(device)
  scores = []

  generate_ids = model(input_ids=input_ids).logits[:,-1,:]

  tensor_softmax = torch.softmax(generate_ids, dim=-1)

  tensor_sum = (tensor_softmax * value_vec).sum(-1)

    

  return tensor_sum

In [ ]:
import torch.optim as optim
import numpy as np

In [ ]:
# simplified version of policy estimate
def get_policy(board_string):
  prompt = """I am the O player in a game tic-tac-toe, the other player is X and I'm supposed to play next. The board configuration is given in input. If I play optimally and the opponent play optimally, reply a single move between 1 to 9 representing what I should play next, with 1 representing A1, 2 representing A2, so on and so forth, and 9 representing C9. No explanations."""
  inputs = generate_prompt(prompt, board_string)
  inputs = tokenizer(inputs, return_tensors="pt", padding=True)
  input_ids = inputs["input_ids"].to(device)
  generate_ids = model(input_ids=input_ids).logits[:,-1,:]

  # Initialize the sum variable to 0
  # result = torch.zeros(1).to(device)
  # for item in scores:
  #   item.requires_grad = True
  # indices_alpha = [29909,29933,29907]
  # indices_alpha2 = [319,350,315]
  # indices_numeric = [29896,29906,29941]
  indices = [29896, 29906, 29941, 29946, 29945, 29953, 29955, 29947, 29929]

  probs = torch.softmax(generate_ids, dim = -1)[:,indices]


  return probs
    

In [ ]:
import torch.nn as nn

In [ ]:
import torch.optim as optim
import tqdm
import time
from torch.autograd import Variable
import logging
import numpy as np

In [ ]:
from tictactoe.TicTacToeGame import TicTacToeGame as Game

In [ ]:
class AverageMeter(object):
    """From https://github.com/pytorch/examples/blob/master/imagenet/main.py"""

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def __repr__(self):
        return f'{self.avg:.2e}'

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class dotdict(dict):
    def __getattr__(self, name):
        return self[name]

In [ ]:
args = dotdict({
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 10,
    'batch_size': 1,
    'cuda': torch.cuda.is_available(),
    'num_channels': 512,
})

In [ ]:
import torch.nn.functional as F


In [ ]:
criterion = nn.CrossEntropyLoss()
mse = nn.MSELoss()

In [ ]:
def loss_pi(targets, outputs):
        # targets += 3
        return criterion(outputs, targets)

def loss_v(targets, outputs):
    # print(targets, outputs, "aaa")

    return mse(outputs.unsqueeze(0),targets.unsqueeze(0))

In [ ]:
def print_board(board_list):
  if board_list.dim() == 3:
    lst = []
    for board in board_list:
          board = board.reshape(3,3)
          n = board.shape[0]
          letters = ['A', 'B', 'C']
          string = '   '
          for y in range(n):
              string += f'{y+1} '
          string += '\n'

          for x in range(n):
              string += f'  {letters[x]} '
              string += '|'
              for y in range(n):
                  string += 'X' * int(board[x, y] == -1)
                  string += 'O' * int(board[x, y] == 1)
                  string += ' ' * int(board[x, y] == 0)
                  string += '|'
              string += '''\n'''
          lst.append(string)
  else:
        board = board_list.reshape(3,3)
        n = board.shape[0]
        letters = ['A', 'B', 'C']
        string = '   '
        for y in range(n):
            string += f'{y+1} '
        string += '\n'

        for x in range(n):
            string += f'  {letters[x]} '
            string += '|'
            for y in range(n):
                string += 'X' * int(board[x, y] == -1)
                string += 'O' * int(board[x, y] == 1)
                string += ' ' * int(board[x, y] == 0)
                string += '|'
            string += '''\n'''
        lst = string
  return lst

In [ ]:
def is_nan(x):
    return x != x

In [ ]:
import torch
import bitsandbytes as bnb
from transformers import GPTNeoForCausalLM
from bitsandbytes.optim import GlobalOptimManager

In [ ]:
accum_iter = 10

In [ ]:
def train(examples):
  optimizer = torch.optim.AdamW(model.parameters(), lr=0.00003, betas=(0.9, 0.995))
  optimizer.zero_grad()
  print(examples,"examples")
  for epoch in range(args.epochs):
      print('EPOCH ::: ' + str(epoch + 1))
      # Actual Training Part


      pi_losses = AverageMeter()
      v_losses = AverageMeter()

      batch_count = int(len(examples) / args.batch_size)

      t = tqdm(range(batch_count), desc='Training Net')
      for (i, _) in enumerate(t):
          
          sample_ids = np.random.randint(len(examples), size=args.batch_size)
          boards, pis, vs = list(zip(*[examples[i] for i in sample_ids]))


          boards = torch.FloatTensor(np.array(boards).astype(np.float64))
          target_pis = torch.FloatTensor(np.array(pis))
          target_vs = torch.FloatTensor(np.array(vs).astype(np.float64))
          target_vs.requires_grad = True
          target_pis.requires_grad = True

          # predict
          if args.cuda:
              boards, target_pis, target_vs = boards.contiguous().cuda(), target_pis.contiguous().cuda(), target_vs.contiguous().cuda()

          # TODO: Convert board to string
          board_string = print_board(boards)

          # compute output
          out_v = get_value(board_string)
          out_pi = get_policy(board_string)
          l_pi = loss_pi(target_pis, out_pi)
          l_v = loss_v(target_vs, out_v)
          total_loss = (l_pi + l_v)
          # total_loss = Variable(total_loss,requires_grad=True)
          print("Target_pi: ", target_pis, "predicted_pi", out_pi, "targetv", target_vs, "predicted_v", out_v, "policy_loss", l_pi, "value_loss", l_v)
          if is_nan(l_pi) or is_nan(l_v):
            "NANS DETECTED !!"
            if is_nan(l_pi):
              l_pi = torch.tensor(0.0, requires_grad=True)
              pi_losses.update(0, boards.size(0))
            if is_nan(l_v):
              l_v = torch.tensor(0.0, requires_grad=True)
            # record loss
            pi_losses.update(l_pi.item(), boards.size(0))
            v_losses.update(l_v.item(), boards.size(0))
            t.set_postfix(Loss_pi=pi_losses, Loss_v=v_losses)
            optimizer.zero_grad()

          else:
            

            # record loss
            pi_losses.update(l_pi.item(), boards.size(0))
            v_losses.update(l_v.item(), boards.size(0))
            t.set_postfix(Loss_pi=pi_losses, Loss_v=v_losses)

            total_loss.backward()

            if (i % 1 == 0) or (i + 1 == len(t)):
              optimizer.step()
              optimizer.zero_grad()

In [ ]:
@torch.no_grad()
def predict(board):
  # timing
  start = time.time()
  #  preparing input
  board = torch.FloatTensor(board.astype(np.float64)).to(device)
  boardstr = print_board(board)
  v = get_value(boardstr)[0]
  pi = get_policy(boardstr)[0]
  return pi.data.cpu().numpy(), v.data.cpu().numpy()

In [ ]:
import logging
import math

import numpy as np

EPS = 5e-7

log = logging.getLogger(__name__)


class MCTS():
    """
    This class handles the MCTS tree.
    """

    def __init__(self, game, args):
        self.game = game
        self.args = args
        self.Qsa = {}  # stores Q values for s,a (as defined in the paper)
        self.Nsa = {}  # stores #times edge s,a was visited
        self.Ns = {}  # stores #times board s was visited
        self.Ps = {}  # stores initial policy (returned by neural net)

        self.Es = {}  # stores game.getGameEnded ended for board s
        self.Vs = {}  # stores game.getValidMoves for board s

    def getActionProb(self, canonicalBoard, temp=1):
        """
        This function performs numMCTSSims simulations of MCTS starting from
        canonicalBoard.

        Returns:
            probs: a policy vector where the probability of the ith action is
                   proportional to Nsa[(s,a)]**(1./temp)
        """
        for i in range(self.args.numMCTSSims):
            self.search(canonicalBoard)

        s = self.game.stringRepresentation(canonicalBoard)
        counts = [self.Nsa[(s, a)] if (s, a) in self.Nsa else 0 for a in range(self.game.getActionSize())]

        if temp == 0:
            bestAs = np.array(np.argwhere(counts == np.max(counts))).flatten()
            bestA = np.random.choice(bestAs)
            probs = [0] * len(counts)
            probs[bestA] = 1
            return probs

        counts = [x ** (1. / temp) for x in counts]
        counts_sum = float(sum(counts))
        probs = [x / counts_sum for x in counts]
        return probs

    def search(self, canonicalBoard):
        """
        This function performs one iteration of MCTS. It is recursively called
        till a leaf node is found. The action chosen at each node is one that
        has the maximum upper confidence bound as in the paper.

        Once a leaf node is found, the neural network is called to return an
        initial policy P and a value v for the state. This value is propagated
        up the search path. In case the leaf node is a terminal state, the
        outcome is propagated up the search path. The values of Ns, Nsa, Qsa are
        updated.

        NOTE: the return values are the negative of the value of the current
        state. This is done since v is in [-1,1] and if v is the value of a
        state for the current player, then its value is -v for the other player.

        Returns:
            v: the negative of the value of the current canonicalBoard
        """

        s = self.game.stringRepresentation(canonicalBoard)

        if s not in self.Es:
            self.Es[s] = self.game.getGameEnded(canonicalBoard, 1)
        if self.Es[s] != 0:
            # terminal node
            return -self.Es[s]

        if s not in self.Ps:
            # leaf node
            self.Ps[s], v = predict(canonicalBoard)
            valids = self.game.getValidMoves(canonicalBoard, 1)
            # print(valids,len(valids),canonicalBoard)
            self.Ps[s] = self.Ps[s] * valids  # masking invalid moves
            sum_Ps_s = np.sum(self.Ps[s])
            if sum_Ps_s > 0:
                self.Ps[s] /= sum_Ps_s  # renormalize
            else:
                # if all valid moves were masked make all valid moves equally probable

                # NB! All valid moves may be masked if either your NNet architecture is insufficient or you've get overfitting or something else.
                # If you have got dozens or hundreds of these messages you should pay attention to your NNet and/or training process.   
                log.error("All valid moves were masked, doing a workaround.")
                self.Ps[s] = self.Ps[s] + valids
                self.Ps[s] /= np.sum(self.Ps[s])

            self.Vs[s] = valids
            self.Ns[s] = 0
            return -v

        valids = self.Vs[s]
        cur_best = -float('inf')
        best_act = -1

        # pick the action with the highest upper confidence bound
        for a in range(self.game.getActionSize()):
            if valids[a]:
                if (s, a) in self.Qsa:
                    u = self.Qsa[(s, a)] + self.args.cpuct * self.Ps[s][a] * math.sqrt(self.Ns[s]) / (
                            1 + self.Nsa[(s, a)])
                else:
                    u = self.args.cpuct * self.Ps[s][a] * math.sqrt(self.Ns[s] + EPS)  # Q = 0 ?

                if u > cur_best:
                    cur_best = u
                    best_act = a

        a = best_act
        next_s, next_player = self.game.getNextState(canonicalBoard, 1, a)
        next_s = self.game.getCanonicalForm(next_s, next_player)

        v = self.search(next_s)

        if (s, a) in self.Qsa:
            self.Qsa[(s, a)] = (self.Nsa[(s, a)] * self.Qsa[(s, a)] + v) / (self.Nsa[(s, a)] + 1)
            self.Nsa[(s, a)] += 1

        else:
            self.Qsa[(s, a)] = v
            self.Nsa[(s, a)] = 1

        self.Ns[s] += 1
        return -v

In [ ]:
import logging
import os
import sys
from collections import deque
from pickle import Pickler, Unpickler
from random import shuffle

import numpy as np
from tqdm import tqdm


log = logging.getLogger(__name__)


class Coach():
    """
    This class executes the self-play + learning. It uses the functions defined
    in Game and NeuralNet. args are specified in main.py.
    """

    def __init__(self, game, args):
        self.game = game
        # self.nnet = nnet
        # self.pnet = self.nnet.__class__(self.game)  # the competitor network
        self.args = args
        self.mcts = MCTS(self.game, self.args)
        self.trainExamplesHistory = []  # history of examples from args.numItersForTrainExamplesHistory latest iterations
        self.skipFirstSelfPlay = False  # can be overriden in loadTrainExamples()

    def executeEpisode(self):
        """
        This function executes one episode of self-play, starting with player 1.
        As the game is played, each turn is added as a training example to
        trainExamples. The game is played till the game ends. After the game
        ends, the outcome of the game is used to assign values to each example
        in trainExamples.

        It uses a temp=1 if episodeStep < tempThreshold, and thereafter
        uses temp=0.

        Returns:
            trainExamples: a list of examples of the form (canonicalBoard, currPlayer, pi,v)
                           pi is the MCTS informed policy vector, v is +1 if
                           the player eventually won the game, else -1.
        """
        trainExamples = []
        board = self.game.getInitBoard()
        self.curPlayer = 1
        episodeStep = 0

        while True:
            episodeStep += 1
            canonicalBoard = self.game.getCanonicalForm(board, self.curPlayer)
            temp = int(episodeStep < self.args.tempThreshold)

            pi = self.mcts.getActionProb(canonicalBoard, temp=temp)
            sym = self.game.getSymmetries(canonicalBoard, pi)
            for b, p in sym:
                trainExamples.append([b, self.curPlayer, p, None])

            action = np.random.choice(len(pi), p=pi)
            board, self.curPlayer = self.game.getNextState(board, self.curPlayer, action)

            r = self.game.getGameEnded(board, self.curPlayer)
            result = []

            if r != 0:
              for x in trainExamples:
                if x[1] != self.curPlayer:
                  win = 1-r
                else:
                  win = r
                result.append((x[0],x[2],win))
              return result

    def learn(self):
        """
        Performs numIters iterations with numEps episodes of self-play in each
        iteration. After every iteration, it retrains neural network with
        examples in trainExamples (which has a maximum length of maxlenofQueue).
        It then pits the new neural network against the old one and accepts it
        only if it wins >= updateThreshold fraction of games.
        """

        for i in range(1, self.args.numIters + 1):
            # bookkeeping
            log.info(f'Starting Iter #{i} ...')
            # examples of the iteration
            if not self.skipFirstSelfPlay or i > 1:
                iterationTrainExamples = deque([], maxlen=self.args.maxlenOfQueue)

                for _ in tqdm(range(self.args.numEps), desc="Self Play"):
                    self.mcts = MCTS(self.game, self.args)  # reset search tree
                    iterationTrainExamples += self.executeEpisode()

                # save the iteration examples to the history 
                self.trainExamplesHistory.append(iterationTrainExamples)

            if len(self.trainExamplesHistory) > self.args.numItersForTrainExamplesHistory:
                log.warning(
                    f"Removing the oldest entry in trainExamples. len(trainExamplesHistory) = {len(self.trainExamplesHistory)}")
                self.trainExamplesHistory.pop(0)
            # backup history to a file
            # NB! the examples were collected using the model from the previous iteration, so (i-1)  
            self.saveTrainExamples(i - 1)

            # shuffle examples before training
            trainExamples = []
            for e in self.trainExamplesHistory:
                trainExamples.extend(e)
            shuffle(trainExamples)

            # # training new network, keeping a copy of the old one
            # self.nnet.save_checkpoint(folder=self.args.checkpoint, filename='temp.pth.tar')
            # self.pnet.load_checkpoint(folder=self.args.checkpoint, filename='temp.pth.tar')


            train(trainExamples)



    def getCheckpointFile(self, iteration):
        return 'checkpoint_' + str(iteration) + '.pth.tar'

    def saveTrainExamples(self, iteration):
        folder = self.args.checkpoint
        if not os.path.exists(folder):
            os.makedirs(folder)
        filename = os.path.join(folder, self.getCheckpointFile(iteration) + ".examples")
        with open(filename, "wb+") as f:
            Pickler(f).dump(self.trainExamplesHistory)
        f.closed

    def loadTrainExamples(self):
        modelFile = os.path.join(self.args.load_folder_file[0], self.args.load_folder_file[1])
        examplesFile = modelFile + ".examples"
        if not os.path.isfile(examplesFile):
            log.warning(f'File "{examplesFile}" with trainExamples not found!')
            r = input("Continue? [y|n]")
            if r != "y":
                sys.exit()
        else:
            log.info("File with trainExamples found. Loading it...")
            with open(examplesFile, "rb") as f:
                self.trainExamplesHistory = Unpickler(f).load()
            log.info('Loading done!')

            # examples based on the model were already collected (loaded)
            self.skipFirstSelfPlay = True

In [ ]:
# Run training loop
log = logging.getLogger(__name__)


args = dotdict({
    'numIters': 1000,
    'numEps': 1,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 25,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,
    'epochs': 1,
     'lr': 0.001,
    'dropout': 0.3,
    'batch_size': 1,
    'cuda': torch.cuda.is_available(),
    'num_channels': 512,

    'checkpoint': './temp/',
    'load_model': False,
    'load_folder_file': ('/dev/models/8x100x50','best.pth.tar'),
    'numItersForTrainExamplesHistory': 20,

})


def main():
    log.info('Loading %s...', Game.__name__)
    g = Game(3)

    # log.info('Loading %s...', nn.__name__)


    log.info('Loading the Coach...')
    c = Coach(g, args)

    if args.load_model:
        log.info("Loading 'trainExamples' from file...")
        c.loadTrainExamples()

    log.info('Starting the learning process 🎉')
    c.learn()


In [ ]:
tokenizer.pad_token = -1

In [ ]:
log.info('Loading %s...', Game.__name__)
g = Game(3)

# log.info('Loading %s...', nn.__name__)


log.info('Loading the Coach...')
c = Coach(g, args)

if args.load_model:
    log.info("Loading 'trainExamples' from file...")
    c.loadTrainExamples()

log.info('Starting the learning process 🎉')
c.learn()

Self Play: 100%|██████████| 1/1 [03:11<00:00, 191.52s/it]


[(array([[ 1, -1,  1],
       [-1, -1,  0],
       [ 0,  0,  0]]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.2647058823529412, 0.08823529411764706, 0.38235294117647056, 0.2647058823529412], 0.001), (array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]]), [0.08333333333333333, 0.041666666666666664, 0.125, 0.08333333333333333, 0.041666666666666664, 0.25, 0.041666666666666664, 0.08333333333333333, 0.25], 0.999), (array([[-1,  0,  0],
       [ 1,  1,  0],
       [-1,  0,  0]]), [0.0, 0.29411764705882354, 0.08823529411764706, 0.0, 0.0, 0.23529411764705882, 0.0, 0.14705882352941177, 0.23529411764705882], 0.999), (array([[ 0,  0,  0],
       [ 0, -1,  0],
       [ 0,  0,  0]]), [0.08, 0.04, 0.16, 0.08, 0.0, 0.28, 0.04, 0.08, 0.24], 0.001), (array([[ 0,  0,  0],
       [ 0,  1,  0],
       [ 0,  0, -1]]), [0.13333333333333333, 0.1, 0.03333333333333333, 0.06666666666666667, 0.0, 0.13333333333333333, 0.2, 0.3333333333333333, 0.0], 0.999), (array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]]), [0.08333333333

Training Net:   0%|          | 0/56 [00:00<?, ?it/s, Loss_pi=2.26e+00, Loss_v=5.39e-01]

Target_pi:  tensor([[0.0000, 0.0000, 0.0000, 0.1471, 0.0000, 0.2941, 0.2353, 0.2353, 0.0882]],
       device='cuda:0', grad_fn=<ToCopyBackward0>) predicted_pi tensor([[0.3889, 0.1224, 0.0815, 0.0616, 0.0404, 0.0247, 0.0257, 0.0269, 0.0417]],
       device='cuda:0', dtype=torch.float16, grad_fn=<IndexBackward0>) targetv tensor([0.9990], device='cuda:0', grad_fn=<ToCopyBackward0>) predicted_v tensor([0.2648], device='cuda:0', grad_fn=<SumBackward1>) policy_loss tensor(2.2616, device='cuda:0', grad_fn=<DivBackward1>) value_loss tensor(0.5390, device='cuda:0', grad_fn=<MseLossBackward0>)


Training Net:   2%|▏         | 1/56 [00:01<01:11,  1.30s/it, Loss_pi=2.27e+00, Loss_v=3.02e-01]

...

Target_pi:  tensor([[0.2647, 0.2647, 0.0000, 0.3824, 0.0000, 0.0000, 0.0882, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ToCopyBackward0>) predicted_pi tensor([[4.7684e-07, 1.2000e-01, 9.2545e-03, 4.9731e-01, 5.4693e-04, 3.6377e-01,
         6.5613e-03, 1.2922e-03, 1.8179e-04]], device='cuda:0',
       dtype=torch.float16, grad_fn=<IndexBackward0>) targetv tensor([0.0010], device='cuda:0', grad_fn=<ToCopyBackward0>) predicted_v tensor([0.5615], device='cuda:0', grad_fn=<SumBackward1>) policy_loss tensor(2.1025, device='cuda:0', grad_fn=<DivBackward1>) value_loss tensor(0.3142, device='cuda:0', grad_fn=<MseLossBackward0>)


Training Net:  13%|█▎        | 29/216 [00:29<03:06,  1.00it/s, Loss_pi=2.23e+00, Loss_v=2.48e-01]

Target_pi:  tensor([[0.0000, 0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9756]],
       device='cuda:0', grad_fn=<ToCopyBackward0>) predicted_pi tensor([[5.9605e-07, 1.1700e-01, 9.7580e-03, 5.2441e-01, 6.1893e-04, 3.3862e-01,
         6.8665e-03, 1.4963e-03, 2.0576e-04]], device='cuda:0',
       dtype=torch.float16, grad_fn=<IndexBackward0>) targetv tensor([0.0010], device='cuda:0', grad_fn=<ToCopyBackward0>) predicted_v tensor([0.5554], device='cuda:0', grad_fn=<SumBackward1>) policy_loss tensor(2.3259, device='cuda:0', grad_fn=<DivBackward1>) value_loss tensor(0.3073, device='cuda:0', grad_fn=<MseLossBackward0>)


Training Net:  14%|█▍        | 30/216 [00:30<03:09,  1.02s/it, Loss_pi=2.23e+00, Loss_v=2.48e-01]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 15>:15                                                                            │
│ in learn:118                                                                                     │
│ in train:37                                                                                      │
│ in get_policy:6                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:1501 in _call_impl             │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/accelerate/hooks.py:165 in new_forward                    │
│                                                                                                  │
│   162 │   │   │   with torch.no_grad():                                                          │
│   163 │   │   │   │   output = old_forward(*args, **kwargs)                                      │
│   164 │   │   else:                                                                              │
│ ❱ 165 │   │   │   output = old_forward(*args, **kwargs)                                          │
│   166 │   │   return module._hf_hook.post_forward(module, output)                                │
│   167 │                                                                                          │
│   168 │   module.forward = new_forward                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/models/llama/modeling_llama.py:687 in        │
│ forward                                                                                          │
│                                                                                                  │
│   684 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return   │
│   685 │   │                                                                                      │
│   686 │   │   # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)    │
│ ❱ 687 │   │   outputs = self.model(                                                              │
│   688 │   │   │   input_ids=input_ids,                                                           │
│   689 │   │   │   attention_mask=attention_mask,                                                 │
│   690 │   │   │   position_ids=position_ids,                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:1501 in _call_impl             │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backwa

In [ ]:
model.save_pretrained("/content/drive/MyDrive/cs229proj/pretraind3", from_pt=True) 